In [1]:
import pandas as pd
from youtube_transcript_api import YouTubeTranscriptApi
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from gensim import corpora
from gensim.models import LdaModel
from gensim.models import CoherenceModel
import string

c:\Users\jorve\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [73]:
def preprocess_text(text):
    tokens = word_tokenize(text)
    tokens = [token.lower() for token in tokens]
    tokens = [token for token in tokens if token not in string.punctuation]

    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]

    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    preprocessed_text = ' '.join(tokens)
    return tokens

In [74]:
in_videos_path = "./trending_videos/USvideos.csv"
in_df = pd.read_csv(in_videos_path)

# Extracting transcripts and preprocessing 
documents = []
documents_directory = []
i = 0
for vid in in_df['video_id']:
    print(vid)
    try:
        transcript_list = YouTubeTranscriptApi.get_transcript(vid)
        transcript_text = ' '.join([item['text'] for item in transcript_list])
        preprocessed_text = preprocess_text(transcript_text)
        documents.append(preprocessed_text)

        i = i + 1
        if i > 10:
            break
    except:
        print("Subtitles are turned off")
    print("")



2kyS6SvSYSE

1ZAPwfrtAFY

5qpjK5DgCt4
Subtitles are turned off

puqaWrEC7tY

d380meD0W0M

gHZ1Qz0KiKM

39idVpFF7NQ

nc99ccSXST0

jr9QtXwC9vc

TUmyygCMMGA

9wRQljFNDW8

VifQlJit6A0
Subtitles are turned off

5E4ZBSInqUU


In [84]:
in_df['video_id'][12]

'5E4ZBSInqUU'

In [76]:
print(documents[10])

['♫♫♫', '♪', 'bass', 'kicking', 'drum', 'drumming', 'beatin', 'block', 'hear', 'coming', '♪', '♪', 'bass', 'kicking', 'drum', 'drumming', 'beatin', 'block', 'hear', 'coming', '♪', '♪', 'bass', 'kicking', 'drum', 'drumming', 'beatin', 'block', 'hear', 'coming', '♪', '♪', 'bass', 'kicking', 'drum', 'drumming', 'beatin', 'block', 'hear', 'coming', '♪', '♪', 'bass', 'kicking', 'drum', 'drumming', 'beatin', 'block', 'hear', 'coming', '♪', '♪', 'bass', 'kicking', 'drum', 'drumming', 'beatin', 'block', 'hear', 'coming', '♪', '♪', 'bass', 'kicking', 'drum', 'drumming', 'beatin', 'block', 'hear', 'coming', '♪', '♪', 'bass', 'kicking', 'drum', 'drumming', 'beatin', 'block', 'hear', 'coming', '♪', '♪', 'bass', 'kicking', 'drum', 'drumming', '♪', '♪', 'bass', 'kicking', 'drum', 'drumming', '♪', '♪', 'bass', 'kicking', 'drum', 'drumming', '♪', '♪', 'bass', 'kicking', 'drum', 'drumming', '♪', '♪', 'bass', 'kicking', 'drum', 'drumming', '♪', '♪', 'bass', 'kicking', '♪', '♪', 'bass', 'kicking', '♪', '

In [4]:
# Create dictionary and corpus
dictionary = corpora.Dictionary(documents)
corpus = [dictionary.doc2bow(doc) for doc in documents]

# Train LDA model
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=5, passes=10)

print("Topics:")
for idx, topic in lda_model.print_topics():
    print(f"Topic {idx}: {topic}")

# Compute coherence score
coherence_model = CoherenceModel(model=lda_model, texts=documents, dictionary=dictionary, coherence='c_v')
coherence_score = coherence_model.get_coherence()
print(f"Coherence Score: {coherence_score}")

Topics:
Topic 0: 0.022*"’" + 0.018*"'s" + 0.014*"♪" + 0.013*"''" + 0.010*"”" + 0.010*"“" + 0.009*"nickelback" + 0.009*"n't" + 0.008*"'re" + 0.008*"``"
Topic 1: 0.029*"'s" + 0.018*"''" + 0.013*"trump" + 0.012*"--" + 0.012*"``" + 0.011*"n't" + 0.011*"audience" + 0.009*"..." + 0.007*"like" + 0.007*"'m"
Topic 2: 0.092*"♪" + 0.046*"bass" + 0.043*"kicking" + 0.034*"drum" + 0.034*"drumming" + 0.022*"hear" + 0.022*"block" + 0.022*"beatin" + 0.022*"coming" + 0.003*"..."
Topic 3: 0.030*"'s" + 0.025*"n't" + 0.020*"like" + 0.011*"'m" + 0.011*"face" + 0.011*"phone" + 0.010*"really" + 0.009*"thing" + 0.009*"point" + 0.008*"video"
Topic 4: 0.032*"'s" + 0.016*"ice" + 0.015*"cream" + 0.014*"like" + 0.014*"'m" + 0.013*"'re" + 0.012*"n't" + 0.010*"let" + 0.009*"going" + 0.009*"got"
Coherence Score: 0.32307516898381333


In [14]:
topics_df = pd.DataFrame(columns=[f"Word_{i}" for i in range(1, 11)])

for idx, topic in lda_model.print_topics():
    words_scores = [word.split("*") for word in topic.split("+")]
    words_scores = [(word.strip(), float(score.split("*")[0])) for score, word in words_scores]
    
    words_scores.sort(key=lambda x: x[1], reverse=True)
    
    row_data = {f"Word_{i}": word_score for i, word_score in enumerate(words_scores, start=1)}
    topics_df = topics_df.append(row_data, ignore_index=True)

topics_df.index = [f"Topic {i}" for i in range(1, len(topics_df) + 1)]

print("Topics DataFrame:")
print(topics_df)

# Compute coherence score
coherence_model = CoherenceModel(model=lda_model, texts=documents, dictionary=dictionary, coherence='c_v')
coherence_score = coherence_model.get_coherence()
print(f"\nCoherence Score: {coherence_score}")

Topics DataFrame:
                Word_1           Word_2              Word_3           Word_4  \
Topic 1   ("’", 0.022)    ("'s", 0.018)        ("♪", 0.014)    ("''", 0.013)   
Topic 2  ("'s", 0.029)    ("''", 0.018)    ("trump", 0.013)    ("--", 0.012)   
Topic 3   ("♪", 0.092)  ("bass", 0.046)  ("kicking", 0.043)  ("drum", 0.034)   
Topic 4   ("'s", 0.03)   ("n't", 0.025)      ("like", 0.02)    ("'m", 0.011)   
Topic 5  ("'s", 0.032)   ("ice", 0.016)    ("cream", 0.015)  ("like", 0.014)   

                      Word_5            Word_6                 Word_7  \
Topic 1          ("”", 0.01)       ("“", 0.01)  ("nickelback", 0.009)   
Topic 2        ("``", 0.012)    ("n't", 0.011)    ("audience", 0.011)   
Topic 3  ("drumming", 0.034)   ("hear", 0.022)       ("block", 0.022)   
Topic 4      ("face", 0.011)  ("phone", 0.011)       ("really", 0.01)   
Topic 5        ("'m", 0.014)    ("'re", 0.013)         ("n't", 0.012)   

                    Word_8             Word_9           Word_1

In [78]:
# Topic Inference

from gensim.models import TfidfModel
tfidf_model = TfidfModel(corpus)

# Transform the corpus to TF-IDF space
corpus_tfidf = tfidf_model[corpus]

# Print TF-IDF 
for doc in corpus_tfidf:
    print(doc)

[(0, 0.015830929186455376), (1, 0.028456952360122183), (2, 0.10963929053698968), (3, 0.04485243703785941), (4, 0.03878018093410353), (5, 0.014950812345953137), (6, 0.026678254257593043), (7, 0.020332975857845062), (8, 0.026678254257593043), (9, 0.03752557932873071), (10, 0.03752557932873071), (11, 0.020332975857845062), (12, 0.015830929186455376), (13, 0.03752557932873071), (14, 0.028456952360122183), (15, 0.03752557932873071), (16, 0.03752557932873071), (17, 0.060998927573535186), (18, 0.03752557932873071), (19, 0.053356508515186085), (20, 0.020332975857845062), (21, 0.015830929186455376), (22, 0.015830929186455376), (23, 0.03752557932873071), (24, 0.11257673798619212), (25, 0.053356508515186085), (26, 0.03752557932873071), (27, 0.15010231731492285), (28, 0.08003476277277913), (29, 0.03752557932873071), (30, 0.03752557932873071), (31, 0.053356508515186085), (32, 0.03166185837291075), (33, 0.03752557932873071), (34, 0.026678254257593043), (35, 0.026678254257593043), (36, 0.026678254257

In [81]:
doc_topic_distribution = [lda_model.get_document_topics(doc_tfidf) for doc_tfidf in corpus_tfidf]

# Map documents to topics
document_topics = []
for doc_topics in doc_topic_distribution:
    # Sort by probability 
    doc_topics.sort(key=lambda x: x[1], reverse=True)
    
    #taking top three topics
    top_three_topics = doc_topics[:3]
    document_topics.append(top_three_topics)


for i, topics in enumerate(document_topics):
    print(f"Document {i} is assigned to topics:")
    for topic, prob in topics:
        print(f"  Topic {topic}: Probability {prob:.3f}")


Document 0 is assigned to topics:
  Topic 4: Probability 0.948
  Topic 3: Probability 0.013
  Topic 1: Probability 0.013
Document 1 is assigned to topics:
  Topic 1: Probability 0.962
Document 2 is assigned to topics:
  Topic 0: Probability 0.931
  Topic 2: Probability 0.017
  Topic 1: Probability 0.017
Document 3 is assigned to topics:
  Topic 3: Probability 0.940
  Topic 4: Probability 0.015
  Topic 1: Probability 0.015
Document 4 is assigned to topics:
  Topic 3: Probability 0.953
  Topic 4: Probability 0.012
  Topic 1: Probability 0.012
Document 5 is assigned to topics:
  Topic 1: Probability 0.935
  Topic 4: Probability 0.016
  Topic 0: Probability 0.016
Document 6 is assigned to topics:
  Topic 4: Probability 0.900
  Topic 3: Probability 0.025
  Topic 1: Probability 0.025
Document 7 is assigned to topics:
  Topic 4: Probability 0.913
  Topic 0: Probability 0.022
  Topic 1: Probability 0.022
Document 8 is assigned to topics:
  Topic 0: Probability 0.925
  Topic 3: Probability 0.01